In [1]:
import glob
import pandas as pd

## Combine all CSVs into one DataFrame

In [2]:
%%time

# Get list of all Actual Load Excel files
path = '../data/raw/MISO - Regional Forecast and Actual Load/test'
allFilePaths = glob.glob(path + "/*.xls")

dfLoad = pd.DataFrame()

for filePath in allFilePaths:
    print filePath
    
    # Import Excel
    df = pd.read_excel(filePath, skiprows=5, skipfooter=3)

    # Filter out the unwanted cells
    df = df.iloc[1:25, 1:]

    # Convert column dtypes to float
    df[df.columns.values[1:]] = df[df.columns.values[1:]].astype(float)

    # Find the daily mean
    #df = df.drop('HourEnding', axis=1).groupby('Market Day').mean()
    
    # Append to main DataFrame
    dfLoad = pd.concat([dfLoad, df])

../data/raw/MISO - Regional Forecast and Actual Load/test\20090707_rf_al.xls
../data/raw/MISO - Regional Forecast and Actual Load/test\20090708_rf_al.xls
../data/raw/MISO - Regional Forecast and Actual Load/test\20090709_rf_al.xls
../data/raw/MISO - Regional Forecast and Actual Load/test\20090710_rf_al.xls
../data/raw/MISO - Regional Forecast and Actual Load/test\20090711_rf_al.xls
../data/raw/MISO - Regional Forecast and Actual Load/test\20090712_rf_al.xls
Wall time: 361 ms


In [4]:
dfLoad

,Market Day,HourEnding,East MTLF (MWh),East ActualLoad (MWh),Central MTLF (MWh),Central ActualLoad (MWh),West MTLF (MWh),West ActualLoad (MWh),Midwest ISO MTLF (MWh),Midwest ISO ActualLoad (MWh)
1,07/06/2009,1,22685,21541.10,16400,16274.62,10123.0,10000.05,49208.0,47815.77
2,07/06/2009,2,22162,20704.83,15643,15590.06,9612.0,9418.52,47417.0,45713.41
3,07/06/2009,3,21839,20323.99,15233,15158.88,9379.0,9141.58,46451.0,44624.45
4,07/06/2009,4,22036,20385.15,15132,15166.17,9310.0,9023.22,46478.0,44574.54
5,07/06/2009,5,23075,21324.51,15747,15635.72,9453.0,9142.20,48275.0,46102.43
6,07/06/2009,6,24646,22598.88,16836,16461.51,9983.0,9579.57,51465.0,48639.96
7,07/06/2009,7,27010,24957.79,17972,17639.37,10979.0,10470.67,55961.0,53067.83
8,07/06/2009,8,28523,27376.44,19380,19275.75,12058.0,11613.78,59961.0,58265.97
9,07/06/2009,9,30026,29405.08,20795,20834.75,12919.0,12498.01,63740.0,62737.84
10,07/06/2009,10,31591,30917.98,22109,22195.76,13560.0,13122.01,67260.0,66235.75


## Write DataFrame to CSV

In [4]:
dfLoad.to_csv('../data/MISOActualLoad.csv')